In [1]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import Sequential
from keras import layers
from keras import backend as K
from keras.layers.core import Dense
from keras import regularizers
from keras.layers import Dropout
from keras.constraints import max_norm

Using TensorFlow backend.


In [2]:
# Import data
train = pd.read_csv('../../01_input/train.csv')
test = pd.read_csv('../../01_input/test.csv')

In [3]:
train.shape

(200000, 202)

In [4]:
test.shape

(200000, 201)

In [5]:
#Check num of cases in label 
print(train.target.value_counts())
print(train.target.value_counts()[1]/train.target.value_counts()[0])

0    179902
1     20098
Name: target, dtype: int64
0.1117163789174106


In [6]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size = 0.25, random_state = 50)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
test_features = sc.transform(test_features)

In [9]:
# Add RUC metric to monitor NN
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [10]:
input_dim = X_train.shape[1]
input_dim

200

In [11]:
# Try early stopping
#from keras.callbacks import EarlyStopping
#callback = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

In [12]:
model = Sequential()
# Input layer
model.add(Dense(units = 200, activation = "relu", input_dim = input_dim, kernel_initializer = "normal", kernel_regularizer=regularizers.l2(0.005), 
                kernel_constraint = max_norm(5.)))
# Add dropout regularization
model.add(Dropout(rate=0.2))

# First hidden layer
model.add(Dense(units = 200, activation='relu', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
model.add(Dropout(rate=0.1))

# Second hidden layer
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
model.add(Dropout(rate=0.1))

# Third hidden layer
model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
model.add(Dropout(rate=0.1))

# Output layer
model.add(layers.Dense(units = 1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)            

In [13]:
model.fit(X_train, y_train, batch_size = 16384, epochs = 125, validation_data = (X_test, y_test))#, callbacks = [callback])

Instructions for updating:
Use tf.cast instead.
Train on 150000 samples, validate on 50000 samples
Epoch 1/125
150000/150000 [==============================] - 2s 13us/step - loss: 2.7544 - acc: 0.8802 - auc: 0.5113 - val_loss: 2.4805 - val_acc: 0.8990 - val_auc: 0.5325
Epoch 2/125
150000/150000 [==============================] - 1s 8us/step - loss: 2.3112 - acc: 0.8997 - auc: 0.5610 - val_loss: 2.0857 - val_acc: 0.8990 - val_auc: 0.5785
Epoch 3/125
150000/150000 [==============================] - 1s 8us/step - loss: 1.9443 - acc: 0.8997 - auc: 0.6013 - val_loss: 1.7423 - val_acc: 0.8990 - val_auc: 0.6322
Epoch 4/125
150000/150000 [==============================] - 1s 8us/step - loss: 1.6279 - acc: 0.8997 - auc: 0.6603 - val_loss: 1.4627 - val_acc: 0.8990 - val_auc: 0.6854
Epoch 5/125
150000/150000 [==============================] - 1s 8us/step - loss: 1.3690 - acc: 0.8997 - auc: 0.7051 - val_loss: 1.2367 - val_acc: 0.8994 - val_auc: 0.7213
Epoch 6/125
150000/150000 [==================

Epoch 47/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2568 - acc: 0.9148 - auc: 0.8466 - val_loss: 0.2576 - val_acc: 0.9132 - val_auc: 0.8468
Epoch 48/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2561 - acc: 0.9148 - auc: 0.8469 - val_loss: 0.2575 - val_acc: 0.9133 - val_auc: 0.8470
Epoch 49/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2559 - acc: 0.9147 - auc: 0.8472 - val_loss: 0.2577 - val_acc: 0.9125 - val_auc: 0.8472
Epoch 50/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2552 - acc: 0.9151 - auc: 0.8474 - val_loss: 0.2569 - val_acc: 0.9133 - val_auc: 0.8475
Epoch 51/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2555 - acc: 0.9149 - auc: 0.8476 - val_loss: 0.2564 - val_acc: 0.9133 - val_auc: 0.8477
Epoch 52/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2551 - acc: 0.9148 - auc: 0.8478 - val_loss: 0.2562 - va

150000/150000 [==============================] - 1s 8us/step - loss: 0.2503 - acc: 0.9150 - auc: 0.8531 - val_loss: 0.2528 - val_acc: 0.9130 - val_auc: 0.8531
Epoch 95/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2501 - acc: 0.9152 - auc: 0.8531 - val_loss: 0.2526 - val_acc: 0.9129 - val_auc: 0.8532
Epoch 96/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2500 - acc: 0.9147 - auc: 0.8532 - val_loss: 0.2533 - val_acc: 0.9136 - val_auc: 0.8533
Epoch 97/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2502 - acc: 0.9152 - auc: 0.8533 - val_loss: 0.2525 - val_acc: 0.9133 - val_auc: 0.8533
Epoch 98/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2498 - acc: 0.9153 - auc: 0.8534 - val_loss: 0.2526 - val_acc: 0.9140 - val_auc: 0.8534
Epoch 99/125
150000/150000 [==============================] - 1s 8us/step - loss: 0.2499 - acc: 0.9150 - auc: 0.8534 - val_loss: 0.2526 - val_acc: 0.9135

In [14]:
y_pred = model.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

0.8602601887866866

In [15]:
id_code_test = test['ID_code']
# Make predicitions
pred = model.predict(test_features)
pred_ = pred[:,0]

In [16]:
pred_

array([0.22856092, 0.26406246, 0.05503333, ..., 0.00544405, 0.04902357,
       0.16328779], dtype=float32)

In [17]:
# To CSV
my_submission = pd.DataFrame({"ID_code" : id_code_test, "target" : pred_})

In [18]:
my_submission

,ID_code,target
0,test_0,0.228561
1,test_1,0.264062
2,test_2,0.055033
3,test_3,0.179801
4,test_4,0.065044
5,test_5,0.006530
6,test_6,0.014584
7,test_7,0.159538
8,test_8,0.006730
9,test_9,0.011020


In [19]:
my_submission.to_csv('../../04_predict_test/242_keras_submission.csv', index = False, header = True)